In [ ]:
import requests
import json

class Spider:
    def __init__(self):
        self.flag = True
        self.url_temp = "https://m.weibo.cn/api/container/getIndex?uid=3881380517&t=0&luicode=10000011&lfid=100103type%3D1%26q%3D%E6%96%B0%E5%8D%8E%E6%97%A5%E6%8A%A5&type=uid&value=3881380517&containerid=1076033881380517"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
        }

    def parse_url(self, url):
        response = requests.get(url, self.headers)
        html_str = response.content.decode()
        return html_str

    def get_content_list(self, html_str):
        content = json.loads(html_str)
        if content['ok'] != 1:
            self.flag = False
        since_id = content['data']['cardlistInfo']['since_id']
        print(since_id)
        content_list = []
        cards = content['data']['cards']
        for c in cards:
            if c['card_type'] == 9:
                temp = {}
                temp['created_at'] = c['mblog']['created_at']
                # 记录文件内容
                t = requests.get("https://m.weibo.cn/statuses/extend?id={}".format(c["mblog"]["id"]), self.headers)
                text = json.loads(t.content.decode())
                temp["longTextContent"] = text['data']['longTextContent']
                content_list.append(temp)

        next_url = "https://m.weibo.cn/api/container/getIndex?uid=3881380517&t=0&luicode=10000011&lfid=100103type%3D1%26q%3D%E6%96%B0%E5%8D%8E%E6%97%A5%E6%8A%A5&type=uid&value=3881380517&containerid=1076033881380517&since_id={}".format(since_id)
        return content_list,next_url

    def run(self):
        content = []
        while self.flag:
            html_str = self.parse_url(self.url_temp)
            content_list,next_url = self.get_content_list(html_str)
            self.url_temp = next_url

            content.extend(content_list)
            print("----------------------------------------------------------------------------------")
            print(content_list)
            

if __name__ == "__main__":
    weibo = Spider()
    weibo.run()